<h1 align="center"> Validating a String of Numeric Input With Reductions and the Ternary Operator </h1>

Python has a variety of methods that will examine a given string and check that it represents a valid number. <br /> ```isdigit(), isnumeric()``` and ```isdecimal()``` are a few examples. 

Let's make a new, more general function that will check if a string is a proper number of any type, i.e., integer, decimal or rational.

How do we know if a given string represents a valid number? Let's start with the integers. The first question to ask is whether or not all the characters in the string are numbers, that is, are they in the set '0123456789'? What about negative numbers? Well, is there a minus sign in the string? We'll need to add '-' to our set so we can check and we now have '-0123456789'.  Further, if there is a minus sign, is there only one and is it at the beginning of our string? Answering these will let us know if our string represents an integer. This can be accomplished in a few lines of Python with a functional programming technique known as reduction. 

A reduction is a higher-order function that uses a combining function to return a single value from a list or set of values. In our example our set will be a string of characters.

A reduction that operates over a simple set or list can be built with a for loop. In the code below, the first loop checks that all characters are in the set '-0123456789'. The second loop checks that if there is a minus sign, it is the first character only; or, said more explicitly, all characters except the first are not '-'. If both of these are true, we will have a string that represents an integer. 


In [100]:
def v(x):
    a, b = True, True
    for i in range(len(x)): a = a and x[i] in "-1234567890."
    for j in range(1, len(x)): b = b and not (x[j] == "-")
    return a and b

print('32', v('32'))
print('3a', v('3a'))
print('-32', v('-32'))
print('32-', v('32-'))

32 True
3a False
-32 True
32- False


We're off to a good start but we can make things more pythonic using generator expressions and the ```all()``` function.

https://www.python.org/dev/peps/pep-0289/#reduction-functions

In [101]:
def v(x):
    a = all(i in '/.-1234567890' for i in x)
    b = all(j != '-' for j in x[1:])
    return a and b

print('32', v('32'))
print('3a', v('3a'))
print('-32', v('-32'))
print('32-', v('32-'))

32 True
3a False
-32 True
32- False


Picking things apart, we can see that the generator expression returns a set of booleans over which the ```all()``` function will operate allowing us to reduce the set to a single value. 

In [102]:
print(list((i in '/.-1234567890' for i in '32-')))
print(list((j != '-' for j in '32-'[1:])))

[True, True, True]
[True, False]


Continuing...

In [103]:
print(all((i in '/.-1234567890' for i in '32-')))
print(all((j != '-' for j in '32-'[1:])))

True
False


Our function tells us that all characters in the string '32-' are in our set of acceptable characters, '-0123456789', but it is not true that all characters except the first are not '-', so ```v('32-')``` returns ```False```. 

Now, what about decimals and floats? We can simply add '.' to our set of acceptable characters, and, as with the minus sign, we will have to make certain that if there is a decimal point, there is one and only one in our string. We can take care of this by counting how many decimal points we have in our string and testing that we have only 0 or 1 of them with an inequality in the ```return``` statement. This can also be done with yet another reduction but now utilizing a different combining function, ```sum()```. As ```sum()``` will tell us how many decimal points we have, we can add ```(c < 2)``` to our ```return``` statement to make certain we have only 0 or 1 of them. 

In [104]:
def v(x):
    a = all(i in '/.-1234567890' for i in x)
    b = all(j != '-' for j in x[1:])
    c = sum(k == '.' for k in x)
    return a and b and (c < 2)

print('32', v('32'))
print('3.2', v('3.2'))
print('3..', v('3..'))

32 True
3.2 True
3.. False


Let's move on to the rational numbers or fractions. As before, we will have a new character, '/', to add to our set. And, just like our other special characters, we will have to make certain that if there is a slash, we have one and only one. Again, we can take care of this by counting how many slashes we have and testing that we have only 0 or 1 of them with an inequality in the ```return``` statement. 

In [105]:
def v(x):
    a = all(i in '/.-1234567890' for i in x)
    b = all(j != '-' for j in x[1:])
    c = sum(k == '.' for k in x)
    d = sum(l == '/' for l in x)
    return a and b and (c < 2) and (d < 2)

print('3/2', v('3/2'))
print('-3/2', v('-3/2'))
print('3//2', v('3//2'))

3/2 True
-3/2 True
3//2 False


Looking good, but if we poke a little we can find a bug, ```v('3./2')``` will evaluate to ```True```. 

In [106]:
print('3./2', v('3./2'))

3./2 True


The problem is that certain combinations of special characters make sense but not all of them; '-3/2' makes sense, '-3.2' makes sense but  a '.' and a '/' cannot be allowed in the same string. We need to strengthen our conditions. We can have one decimal point or none and we can have one slash or none, but not both. Let's examine things further with a truth table. 

"."|"-"| v
---|---|---
 0 | 0 | T
 1 | 0 | T
 0 | 1 | T
 1 | 1 | F

This is the truth table for NAND, or ```not and ```, and we can easily tweak our return statement to fix the bug.

In [107]:
def v(x):
    a = all(i in '/.-1234567890' for i in x)
    b = all(j != '-' for j in x[1:])
    c = sum(k == '.' for k in x)
    d = sum(l == '/' for l in x)
    return a and b and not ((c < 2) and (d < 2))

print(v('3./2'))

False


Great! But we can make things a little more pythonic by summing the rows of our truth table and seeing what happens.

"."|"-"|sum
---|---|---
 0 | 0 | 0
 1 | 0 | 1
 0 | 1 | 1
 1 | 1 | 2
 
We can see that by counting the total number of minus signs and decimal points we can check that we don't have both at the same time with the same inequality,```< 2``` , that ensures that we don't have more than two of either one of them. Now we can make our return statement more concise. 

In [108]:
def v(x):
    a = all(i in '/.-1234567890' for i in x)
    b = all(j != '-' for j in x[1:])
    c = sum(k == '.' for k in x)
    d = sum(l == '/' for l in x)
    return a and b and (c + d < 2)

print(v('3./2'))

False


Let's look for more bugs. What about ```v('.')```, ```v('-')``` or ```v('/')```?

In [109]:
print('.', v('.'))
print('-', v('-'))
print('/', v('/'))

. True
- True
/ True


We can easily take care of the first two in one line with a pair of reductions and a little logic. The question we need to answer is, if there is one character in the string is it a decimal point or minus sign? The answer needs to be no in order to invalidate the string. Another way of asking this would be, is it not true that all the characters are '.' or '-'?

In [110]:
def v(x):
    a = all(i in '/.-1234567890' for i in x)
    b = all(j != '-' for j in x[1:])
    c = sum(k == '.' for k in x)
    d = sum(l == '/' for l in x)
    e = not (all(m in '-' for m in x) or all(m in '.' for m in x))
    return a and b and (c + d < 2) and e

print('.', v('.'))
print('-', v('-'))

. False
- False


Moving on to ```v('/')``` we will encounter a few more issues. Since a slash means we are dealing with rational numbers or fractions, '/' cannot be the first or last character and  we cannot have a zero, or zeros, in the denominator. Again, we can make short work of things but this time with the ternary operator in the form of a python dictionary where the keys are booleans, ```True``` and ```False```, and the values are lambda functions. 

The ternary operator is simply another form of conditional expression equivalent to ```if else``` statements but in a more compact syntax.

https://www.geeksforgeeks.org/ternary-operator-in-python/

First, let's check to make certain that neither the first nor the last characters are a slash, '/'. In doing so, we will also catch the bug that occurs when a slash is the only character, ```v('/')```.


As was stated earlier, we will be working with a ternary operator in the form of a dictionary that evaluates lambda functions. 

```{True: lambda 1, False: lambda 2}[condition](string to evaluate)```

Our first condition will be whether or not the length of our string is greater than zero.

```{True: lambda 1, False: lambda 2}[len(x) != 0](x) ```

If it is greather than zero, ```True``` will give us our first lambda function that will check to make certain neither the first nor the last characters are a slash, '/':

```lambda x: x[0] != '/' and x[len(x) - 1] != '/'```

If we happen to have a string with a single character, then that character is both the first and last character in the string.<br /> 
Therefore, ```x[0] != '/'``` is the same as ```x[len(x) - 1] != '/'``` thus solving the ```v('/')``` bug.


In [176]:
print('3/2', {True: lambda x: x[0] != '/' and x[len(x) - 1] != '/', False: lambda x: x != ''}[len('3/2') != 0]('3/2'))

print('32/', {True: lambda x: x[0] != '/' and x[len(x) - 1] != '/', False: lambda x: x != ''}[len('32/') != 0]('32/'))

print('/32', {True: lambda x: x[0] != '/' and x[len(x) - 1] != '/', False: lambda x: x != ''}[len('/32') != 0]('/32'))

print('/'  , {True: lambda x: x[0] != '/' and x[len(x) - 1] != '/', False: lambda x: x != ''}[len('/') != 0]('/'))

3/2 True
32/ False
/32 False
/ False


This portion of our code will catch yet another bug, ```v('')```, an empty string. It is dealt with by our second lambda function, ```lambda x: x != ''```, as returned by ```False```.  


In [112]:
print('\'\'', {True: lambda x: x[0] != '/' and x[len(x) - 1] != '/', False: lambda x: x != ''}[len('') != 0](''))

'' False


Finally, if we have a fraction, we need to make certain our denominator is not zero. We will again use a ternary operator in the form of a python dictionary that returns lambda functions.

If we have a slash, '/' we will need to check for zero.

```{True: lambda 1, False: lambda 2}[number of '/' == 1](x) ```

So, if we have a slash, ```True``` will return ```lambda x: not all(n == '0' for n in x.split('/')[1])```

Here, we split on the slash and use another reduction to make certain that all the characters in the denominator are not zeros.

As our conditional checks that the number of '/' == 1, we have two other possibilities. If the number of '/' == 0, ```False``` will evaluate our second function ```lambda x: x != ''``` which returns ```True``` so long as the string is not empty. If the number of '/' > 1 then our inequality in the ```return``` statement will invalidate the string.

We are now ready to put everything together into the final function.

In [175]:
def v(x):
    a = all(i in '/.-1234567890' for i in x)
    b = all(j != '-' for j in x[1:])
    c = sum(k == '.' for k in x)
    d = sum(l == '/' for l in x)
    e = not (all(m in '-' for m in x) or all(m in '.' for m in x))
    f = {True: lambda x: x[0] != '/' and x[len(x) - 1] != '/',
        False: lambda x: x != ''}[len(x) != 0](x)
    g = {True: lambda x: not all(n == '0' for n in x.split('/')[1]), 
        False: lambda x: x != ''}[d == 1](x)
    return a and b and (c + d < 2) and e and f and g

nums = ['32', '3a', '3.2', '3/2', '-32', '-3.2', '-3/2', '--3/2', '/32', '32/', '3./2', 
           '3..2', '3//2', '0/3', '3/0', '3/00', '3/20', '3/02', '/', '-', '.', '']

for i in nums: print(i, v(i))

32 True
3a False
3.2 True
3/2 True
-32 True
-3.2 True
-3/2 True
--3/2 False
/32 False
32/ False
3./2 False
3..2 False
3//2 False
0/3 True
3/0 False
3/00 False
3/20 True
3/02 True
/ False
- False
. False
 False
